In [1]:
import pandas as pd
import numpy as np
import datetime
from filegenerators import *
pd.options.display.float_format = '{:.2e}'.format
import os
import bibtexparser
from numpy.typing import NDArray

In [2]:
class Theoretical_Ranges:
    def __init__(self, min_max_path: str, input_names: list[str], must_be_zero: list[str],
                 scaling_factor: float=1e-12, sheet_name:str = 'icranges'):
        self.name = os.path.splitext(os.path.basename(min_max_path))[0]
        self.df_ranges = pd.read_excel(min_max_path, sheet_name=sheet_name)
        self.df_ranges['value'] = self.df_ranges['value'].astype(float)
        self.df_ranges.species = [s.upper() for s in self.df_ranges.species]
        self.scaling_factor = scaling_factor
        self.df_scaled_ranges = self.df_ranges.select_dtypes(include='number') * self.scaling_factor
        self.must_be_zero = must_be_zero
        self.input_names = input_names
        self.get_input_data(input_names)
        self.bounds = self.get_bounds()
        self.gen_lookuptable()

    def get_input_data(self, input_names: list[str]) -> None:
        self.inputs = {}
        for i in input_names:
            self.inputs[i] = 0.0
        self.inputs["REF"] = 1.0
        self.inputs["Insulin"] = 1e-10

        self.input_data = pd.DataFrame([
            {'species': species, 'minconc': value*1e+12, 'value': value*1e+12, 'maxconc': value*1e+12} # WHAT???
            for species, value in self.inputs.items()])

    def gen_lookuptable(self) -> None:
        self.lut = pd.concat([self.df_ranges, self.input_data], ignore_index=True)    # look-up-table
        self.lut['species'] = self.lut['species'].str.upper()
        print(f"LUT was created successfully. Its dimensions are: {self.lut.shape}")

    def get_bounds(self) -> dict[str, tuple[float, float]]:
        bounds = dict()
        for index, row in self.df_ranges.iterrows():
            if row.value < 0.1:
                lb = 1e-14
                ub = 1e-13
            else:
                lb = (row.value/2)*1e-12
                ub = (row.value*1.5)*1e-12
            bounds[row.species.upper()] = [lb, ub]
        print(bounds['TBID'])
        for index, row in self.input_data.iterrows():
            if row.species.upper() not in bounds.keys():
                bounds[row.species] = [row.minconc*1e-12, row.maxconc*1e-12]
        for m in self.must_be_zero:
            bounds[m.upper()] = [0, 0]
        return bounds


In [ ]:
class Experiment:
    def __init__(self, data_source, # data_source: a pandas DataFrame
                 stresses: float, species_rng: Theoretical_Ranges,
                 sheet_name: str, bibtex: str = ""):
        self.rng = species_rng
        if isinstance(data_source, pd.DataFrame): # Ha pandas DataFrame (azaz xlsx worksheet)
            self.name = sheet_name
            self.experiment_data = data_source.copy()
        else:
            raise ValueError("data_source must be a pandas DataFrame")

        self.stresses = stresses
        self.bibtex = self.parse_bibtex(bibtex)
        self.non_species_cols = {"TIME"}
        self.process_data()

    def parse_bibtex(self, bibtex_str):
        parser = bibtexparser.loads(bibtex_str)
        if not parser.entries:
            raise ValueError("No valid BibTeX entry found.")
        entry = parser.entries[0]  # Assume only one entry is given

        return {
            "author": entry.get("author", ""),
            "title": entry.get("title", ""),
            "journal": entry.get("journal", ""),
            "volume": entry.get("volume", ""),
            "number": entry.get("number", ""),
            "year": entry.get("year", ""),
            "doi": entry.get("doi", entry.get("url", ""))  # fallback if no DOI
        }

    def process_data(self) -> None:
        self.experiment_data.columns = [col.upper() for col in self.experiment_data.columns]
        self.experiment_data.rename(columns={'TIME': 'time'}, inplace=True)
        self.experiment_data.time = self.experiment_data.time * 60  # Converting hrs to mins - if I convert, some xmls fail for some reason
        self.experiment_data = self.experiment_data.dropna()
        self.species = [v for v in self.experiment_data.columns if v.upper() not in self.non_species_cols and "STD" not in v.upper()]

    def get_std(self, s: str, quant_Data: pd.DataFrame, wide=False):
        min = np.min(quant_Data[s])
        max = np.max(quant_Data[s])
        if wide:
            s_std = ((max-min)/8)*1e-12
        else:
            s_std = ((row.value*1.5-row.value/2)/8)*1e-12
        # insert column right after column s in quant_Data. All the values in the column = s_std

    def quantitated_exp_data(self, ics: dict[str, float]) -> None:
        quant_Data = self.experiment_data.copy()
        species_and_std = [col for col in quant_Data.columns if col.upper() not in self.non_species_cols]
        # What if no std is given --> put in formula stuff
        for s in species_and_std:
            if 'STD' not in s.upper() and f'{s}STD' not in species_and_std:
                self.get_std(s, quant_Data)
            if 'STD' in s.upper():
                quant_Data[s] *= ics[s[0:-3]] # scaling the standard dev.
            else:
                quant_Data[s] *= ics[s]
        self.quant_data = quant_Data

    def check_compatibility(self) -> None:
        for s in self.species:
            if s.upper() not in self.rng.bounds.keys():
                self.rng.bounds[s] = [0, 0]
                print(f"Creating new entry for {s}\n")
            #else:
            #    print('All compatible\n')


In [9]:
exp.quant_data

,time,AUT,AUTSTD,PHAG,PHAGSTD
0,0.00e+00,9.26e-14,0.00e+00,7.16e-14,0.00e+00
1,1.50e+01,9.83e-14,4.91e-15,6.85e-14,4.92e-15
2,3.00e+01,1.02e-13,1.05e-14,6.43e-14,4.18e-15
3,4.50e+01,9.90e-14,1.07e-14,5.97e-14,4.82e-15
4,6.00e+01,1.00e-13,1.14e-14,5.99e-14,3.51e-15
5,7.50e+01,1.01e-13,6.84e-15,5.96e-14,2.81e-15
6,9.00e+01,9.88e-14,9.99e-15,6.06e-14,1.59e-15
7,1.05e+02,9.78e-14,7.90e-15,6.03e-14,1.78e-15
8,1.20e+02,9.51e-14,8.63e-15,5.98e-14,1.99e-15
9,1.50e+02,9.22e-14,7.59e-15,6.03e-14,2.45e-15


In [4]:
class Simulation:
    def __init__(self, species_range: Theoretical_Ranges, experiment: Experiment, max_digit = 4):
        self.species_range = species_range
        self.experiment = experiment
        self.maxdigit = max_digit
        self.experiment.check_compatibility()

    def create_xml_files(self, output_xmls_path: str, num_of_xmls: int, xml_template_path: str) -> None:
        if not os.path.exists(output_xmls_path):
            os.makedirs(output_xmls_path)
        self.num_of_xmls = num_of_xmls
        env = jinja2.Environment(loader=jinja2.FileSystemLoader(os.path.dirname(xml_template_path)))
        self.template = env.get_template(os.path.basename(xml_template_path))

        for i in range(1, num_of_xmls+1):
            np.random.seed(i)
            self.random_ics = self.get_random_ics()
            self.experiment.quantitated_exp_data(ics=self.random_ics)
            self.make_xml_output(i, output_xmls_path)

    def get_random_ics(self) -> dict[str, float]:
        random_ics = {}
        #random_ics = {s: np.random.uniform(*self.species_range.bounds[s]) for s in self.species_range.bounds}
        inp_upper = [s.upper() for s in self.species_range.input_names]
        mbz_upper = [s.upper() for s in self.species_range.must_be_zero]
        for key in self.species_range.bounds:
            if self.species_range.bounds[key][0] == 0 and self.species_range.bounds[key][1] == 0 and key not in inp_upper and key not in mbz_upper:
                random_ics[key] = 1e-13
            else:
                random_ics[key] = np.random.uniform(*self.species_range.bounds[key])
        random_ics['RAP'] = self.experiment.stresses
        random_ics["REF"] = 1.0
        return random_ics

    def make_xml_output(self, file_index: int, output_xmls_path: str) -> None:
        dataPoints = [self.compileDataRow(row.values) for _, row in self.experiment.quant_data.iterrows()]
        output = self.template.render(ics=self.random_ics, variables=self.experiment.species, dataPoints=dataPoints, bib=self.experiment.bibtex)
        padded_number = str(file_index).zfill(self.maxdigit)
        filename = f"stressful_life_{padded_number}.xml"
        with open(os.path.join(output_xmls_path, filename), 'w') as f:
            f.write(output)

    def compileDataRow(self, dataPoints):
        meas = "".join(f"<{v}>{{:.4e}}</{v}>" for v in self.experiment.experiment_data.columns)
        return f"<dataPoint>{meas.format(*dataPoints)}</dataPoint>"

    def generate_opp_content(self, xml_folder: str, name: str, mech_file: str,
                             yaml_file: str, time_limit: int, thread_limit: int,
                             settings_tag: str, solver: str) -> str:

      # Create MECHMOD section
      mechmod = f"""MECHMOD
      USE_NAME         BCRN6
      MECH_FILE        {mech_file}
      COMPILE_cantera  {yaml_file}
      END
      """

      # Create MECHTEST section
      mechtest = f"""MECHTEST
      MECHANISM  BCRN6
      TIME_LIMIT {time_limit}
      THREAD_LIMIT {thread_limit}
      SETTINGS_TAG {settings_tag}
      FALLBACK_TO_DEFAULT_SETTINGS

      SOLVER {solver}
      SAVE_STATES      CSV
      """

      # Add each XML file name
      for xml_idx in range(1, self.num_of_xmls+1):
          padded_number = str(xml_idx).zfill(self.maxdigit)
          mechtest += f"      NAME {xml_folder}/stressful_life_{padded_number}.xml\n"

      mechtest += "END\n"

      return mechmod + "\n" + mechtest

    def gen_opp(self, output_dir, sheet_name, opp_output_dir, mech_file: str = "7_Krisztian/mech/BCRN6.inp",
                yaml_file: str = "7_Krisztian/mech/BCRN6.yaml", time_limit: int = 50, thread_limit: int = 32,
                settings_tag: str = "systems_biology", solver: str = "cantera"):
        date = datetime.datetime.now()
        opp_content = self.generate_opp_content(output_dir, sheet_name, mech_file, yaml_file, time_limit,
                                                thread_limit, settings_tag, solver)  # Create .opp file content
        opp_filename = f"{date.year}{date.month}{date.day}_BCRN_{self.experiment.bibtex['author'].split()[0][:-1]}_{sheet_name}.opp" # Define output .opp file path
        with open(os.path.join(opp_output_dir, opp_filename), "w") as f:
            f.write(opp_content)


In [5]:
# 1e-12 scaling for mol/cm^3
# uniformly dist. rap of [rap_input] nano mol     NOT     mol/dm^3 (= nM) cc.
# in a volume of V = 10e-12 dm^3
# uniformly "absorbed" by 2*10^5 cells
rap_input = np.array([100, 50, 10, 5, 1, 0.5, 0.1, 0]) * 1e-12  # Converting to mol/cm^3 (=mol/mL)
rap_in_cells = rap_input / (200000 * 1e-12 * 1e+3)  # rap_in_well_mol / (cell_num * V_cell_in_L * conversion_factor_to_mL)
stress1 = 'rap'
stresses: dict[str, NDArray[np.float_]] = {stress1: rap_in_cells}
stresses

{'rap': array([5.0e-07, 2.5e-07, 5.0e-08, 2.5e-08, 5.0e-09, 2.5e-09, 5.0e-10,
        0.0e+00])}

In [6]:
input_names = ['nS', 'RAP', 'TG', 'dS', 'CCH', 'REF', 'Insulin', 'TG_SERCA', 'RKMTORA', 'casp', 'IP3R', 'Baxa', 'tBid']
must_be_zero = ['casp', 'Baxa', 'tBid', 'p53a', 'PUMA']

rng = Theoretical_Ranges('/home/nvme/Opt/7_Krisztian/0_evaluate/input_files/reactions_ics_finalised.xlsx',
                         input_names, must_be_zero)

[1e-14, 1e-13]
LUT was created successfully. Its dimensions are: (85, 4)


In [7]:
exp_xlsx_path = '/home/nvme/Opt/7_Krisztian/0_evaluate/input_files/Nitin_rap.xlsx'
xml_template = '/home/nvme/Opt/7_Krisztian/0_evaluate/input_files/xml_template.xml'

# Parse stresses

# Read all sheets from Excel
all_sheets = pd.read_excel(exp_xlsx_path, sheet_name=None)  # dict of {sheet_name: DataFrame}

# Extract BibTeX from the last sheet
last_sheet_name = list(all_sheets.keys())[-1]
bibtex_df = all_sheets[last_sheet_name]
# Ha nem lenne header a BibTex-nel, akk ezzel kell beolvasni a sheetet: bibtex_df = pd.read_excel(exp_xlsx_path, sheet_name=last_sheet_name, header=None)

# Join all non-empty strings from the first column into a BibTeX string
bibtex_lines = bibtex_df.iloc[:, 0].dropna().astype(str).tolist()
bibtex_str = "\n".join(bibtex_lines)
bibtex_str = "\n".join(bibtex_lines).replace("_x000d_", "")  # Clean malformed carriage returns

#print("BibTex:\n", bibtex_str, "\n")
opp_output_dir = '/home/nvme/Opt/7_Krisztian/1_mechtest'
num_xml = 15

for i, sheet_name in enumerate(list(all_sheets.keys())[:-1]):
    df = all_sheets[sheet_name]
    exp = Experiment(df, stresses['rap'][i], rng, sheet_name, bibtex_str)
    output_dir = f"/home/nvme/Opt/7_Krisztian/xml/{exp.bibtex['author'].split()[0][:-1]}_{exp.bibtex['year']}/{exp.name}"
    sim = Simulation(rng, exp)
    sim.create_xml_files(output_dir, num_xml, xml_template)
    sim.gen_opp(output_dir, sheet_name, opp_output_dir, mech_file="7_Krisztian/mech/BCRN_Nitin_rap.inp",
                yaml_file="7_Krisztian/mech/BCRN_Nitin_rap.yaml")
    

In [ ]:
# I have to be in the folder where 'bin' is, or rewrite the commands below!!!!!!

# bin/Release/OptimaPP 7_Krisztian/1_mechtest/2025625_BCRN_Beesabathuni_Rap_100_nM.opp